In [2]:
#imports for all exercises
import nltk
from nltk import grammar, parse

# Natural Language Processing

## Exercise Sheet 9

### Exercise 1

Take the following grammar:

In [2]:
g = """
%start S
S                    -> NP[AGR=?n] VP[AGR=?n]
NP[AGR=?n]           -> PropN[AGR=?n]
VP[TENSE=?t, AGR=?n] -> Cop[TENSE=?t, AGR=?n] Adj


Cop[TENSE=pres,  AGR=[NUM=sg, PER=3]] -> 'is'
Cop[TENSE=pres,  AGR=[NUM=sg, PER=1]] -> 'am'
Cop[TENSE=pres,  AGR=[NUM=sg, PER=2]] -> 'are'
Cop[TENSE=pres,  AGR=[NUM=pl, PER=1]] -> 'are'
PropN[AGR=[NUM=sg, PER=3]]            -> 'she'
PropN[AGR=[NUM=sg, PER=1]]            -> 'I'
PropN[AGR=[NUM=sg, PER=2]]            -> 'you'
PropN[AGR=[NUM=pl, PER=1]]            -> 'they'

Adj                                   -> 'happy'
"""
gr = grammar.FeatureGrammar.fromstring(g)

def parse_sent(sent, gr):
    tokens = sent.split()
    parser = parse.FeatureEarleyChartParser(gr)
    trees = parser.parse(tokens)
    for tree in trees: print(tree)

as starting point to correctly parse word sequences like "I am happy" and "she is happy" but not " * you is happy" or " * they am happy". 

In [3]:
sent1 = "I am happy"
sent2 = "she is happy"
sent3 = "you is happy"
sent4 = "they am happy"

In [4]:
parse_sent(sent1, gr)
parse_sent(sent3, gr)
parse_sent(sent4, gr)
parse_sent(sent2, gr)

(S[]
  (NP[AGR=[NUM='sg', PER=1]] (PropN[AGR=[NUM='sg', PER=1]] I))
  (VP[AGR=[NUM='sg', PER=1], TENSE='pres']
    (Cop[AGR=[NUM='sg', PER=1], TENSE='pres'] am)
    (Adj[] happy)))
(S[]
  (NP[AGR=[NUM='sg', PER=3]] (PropN[AGR=[NUM='sg', PER=3]] she))
  (VP[AGR=[NUM='sg', PER=3], TENSE='pres']
    (Cop[AGR=[NUM='sg', PER=3], TENSE='pres'] is)
    (Adj[] happy)))


### Exercise 2

Develop a variant of the following grammar

In [5]:
g = """
% start S

# ###################
# Grammar Productions
# ###################
# S expansion productions
S -> NP[NUM=?n] VP[NUM=?n]
# NP expansion productions
NP[NUM=?n] -> N[NUM=?n, -count]
NP[NUM=?n] -> PropN[NUM=?n]
NP[NUM=?n] -> Det[NUM=?n] N[NUM=?n]
NP[NUM=pl] -> N[NUM=pl]
# VP expansion productions
VP[TENSE=?t, NUM=?n] -> IV[TENSE=?t, NUM=?n]
VP[TENSE=?t, NUM=?n] -> TV[TENSE=?t, NUM=?n] NP
VP[TENSE=?t, NUM=?n] -> Cop[TENSE=?t, NUM=?n] Adj

# ###################
# Lexical Productions
# ###################
Det[NUM=sg] -> 'this' | 'every'
Det[NUM=pl] -> 'these' | 'all'
Det -> 'the' | 'some' | 'several'
PropN[NUM=sg]-> 'Kim' | 'Jody'
N[NUM=sg, +count] -> 'boy' | 'dog' | 'girl' | 'car' | 'child'
N[NUM=pl, +count] -> 'boys' | 'dogs' | 'girls' | 'cars' | 'children'
N[NUM=sg, -count] -> 'water'
IV[TENSE=pres, NUM=sg] -> 'disappears' | 'walks' | 'sings'
TV[TENSE=pres, NUM=sg] -> 'sees' | 'likes'
IV[TENSE=pres, NUM=pl] -> 'disappear' | 'walk' | 'sing'
TV[TENSE=pres, NUM=pl] -> 'see' | 'like'
IV[TENSE=past] -> 'disappeared' | 'walked'
TV[TENSE=past] -> 'saw' | 'liked'
Cop[TENSE=pres, NUM=sg] -> 'is'
Adj -> 'precious'
"""
gr = grammar.FeatureGrammar.fromstring(g)

def parse_sent(sent, gr):
    tokens = sent.split()
    parser = parse.FeatureEarleyChartParser(gr)
    trees = parser.parse(tokens)
    for tree in trees: print(tree)

that uses a feature `COUNT` to make the distinctions shown below:

(1a) the boy sings

(1b) * boy sings

(2a) the boys sing

(2b) boys sing

(3a) the water is precious

(3b) water is precious

In [6]:
sent1 = "the boy sings"
sent2 = "boy sings"
sent3 = "the boys sing"
sent4 = "boys sing"
sent5 = "the water is precious"
sent6 = "water is precious"

In [7]:
parse_sent(sent1, gr)
parse_sent(sent2, gr)
parse_sent(sent3, gr)
parse_sent(sent4, gr)
parse_sent(sent5, gr)
parse_sent(sent6, gr)


(S[]
  (NP[NUM='sg'] (Det[] the) (N[NUM='sg', +count] boy))
  (VP[NUM='sg', TENSE='pres'] (IV[NUM='sg', TENSE='pres'] sings)))
(S[]
  (NP[NUM='pl'] (Det[] the) (N[NUM='pl', +count] boys))
  (VP[NUM='pl', TENSE='pres'] (IV[NUM='pl', TENSE='pres'] sing)))
(S[]
  (NP[NUM='pl'] (N[NUM='pl', +count] boys))
  (VP[NUM='pl', TENSE='pres'] (IV[NUM='pl', TENSE='pres'] sing)))
(S[]
  (NP[NUM='sg'] (Det[] the) (N[NUM='sg', -count] water))
  (VP[NUM='sg', TENSE='pres']
    (Cop[NUM='sg', TENSE='pres'] is)
    (Adj[] precious)))
(S[]
  (NP[NUM='sg'] (N[NUM='sg', -count] water))
  (VP[NUM='sg', TENSE='pres']
    (Cop[NUM='sg', TENSE='pres'] is)
    (Adj[] precious)))


### Exercise 3

Extend the German grammar

so that it can handle so-called verb-second structures like "heute sieht der Hund die Katze" by using a slash category `S/TV` for the missing transitive verb in "der Hund die Katze". Use the following test sentences:

In [18]:
sent1 = "heute sieht der Hund die Katze"
sent2 = "heute sehe der Hund die Katze"
sent3 = "heute sieht der Hund die Katzen"

### Exercise 4

Consider the patterns of grammaticality for the verbs "loaded", "filled", and "dumped" below. Write grammar productions to handle such data:

(1a) the farmer loaded the cart with sand

(1b) the farmer loaded sand into the cart

(2a) the farmer filled the cart with sand

(2b) * the farmer filled sand into the cart

(3a) * the farmer dumped the cart with sand

(3b) the farmer dumped sand into the cart

In [2]:
g = """
% start S
# Grammar Productions
S -> NP[NUM=?n] VP[NUM=?n]
NP[NUM=?n] -> Det[NUM?=n] N[NUM=?n]
NP[NUM=sg] -> N[NUM=sg]
NP[NUM=sg] ->PropN[NUM=sg]
VP[TENSE=past, NUM=sg] -> IV[TENSE=past, NUM=sg] NP PREP[CASE=loaded] NP
VP[TENSE=past, NUM=sg] -> TV[TENSE=past, NUM=sg] NP PREP[CASE=filled] NP
VP[TENSE=past, NUM=sg] -> NV[TENSE=past, NUM=sg] NP PREP[CASE=dumped] NP
#Lexical productions
Det -> 'the' 
N[NUM=sg] -> 'farmer' | 'cart' 
PropN[NUM=sg] ->'sand'

IV[TENSE=past, NUM=sg] ->'loaded'
TV[TENSE=past, NUM=sg] ->'filled'
NV[TENSE=past, NUM=sg] ->'dumped'

PREP[CASE=loaded] -> 'with' | 'into'
PREP[CASE=filled] -> 'with' 
PREP[CASE=dumped] ->  'into'

"""

gr = grammar.FeatureGrammar.fromstring(g)

def parse_sent(sent, gr):
    tokens = sent.split()
    parser = parse.FeatureEarleyChartParser(gr)
    trees = parser.parse(tokens)
    for tree in trees: print(tree)
sent1 = "the farmer loaded the cart with sand"
sent2 = "the farmer loaded sand into the cart"
sent3 = "the farmer filled the cart with sand"
sent4 = "the farmer filled sand into the cart"
sent5 = "the farmer dumped the cart with sand"
sent6 = "the farmer dumped sand into the cart"

In [3]:
parse_sent(sent1, gr)
parse_sent(sent2, gr)
parse_sent(sent3, gr)
parse_sent(sent4, gr)
parse_sent(sent5, gr)
parse_sent(sent6, gr)

(S[]
  (NP[NUM='sg'] (Det[] the) (N[NUM='sg'] farmer))
  (VP[NUM='sg', TENSE='past']
    (IV[NUM='sg', TENSE='past'] loaded)
    (NP[NUM='sg'] (Det[] the) (N[NUM='sg'] cart))
    (PREP[CASE='loaded'] with)
    (NP[NUM='sg'] (PropN[NUM='sg'] sand))))
(S[]
  (NP[NUM='sg'] (Det[] the) (N[NUM='sg'] farmer))
  (VP[NUM='sg', TENSE='past']
    (IV[NUM='sg', TENSE='past'] loaded)
    (NP[NUM='sg'] (PropN[NUM='sg'] sand))
    (PREP[CASE='loaded'] into)
    (NP[NUM='sg'] (Det[] the) (N[NUM='sg'] cart))))
(S[]
  (NP[NUM='sg'] (Det[] the) (N[NUM='sg'] farmer))
  (VP[NUM='sg', TENSE='past']
    (TV[NUM='sg', TENSE='past'] filled)
    (NP[NUM='sg'] (Det[] the) (N[NUM='sg'] cart))
    (PREP[CASE='filled'] with)
    (NP[NUM='sg'] (PropN[NUM='sg'] sand))))
(S[]
  (NP[NUM='sg'] (Det[] the) (N[NUM='sg'] farmer))
  (VP[NUM='sg', TENSE='past']
    (NV[NUM='sg', TENSE='past'] dumped)
    (NP[NUM='sg'] (PropN[NUM='sg'] sand))
    (PREP[CASE='dumped'] into)
    (NP[NUM='sg'] (Det[] the) (N[NUM='sg'] cart))))


### Exercise 5

Consider the following feature structures:

In [15]:
fs1 = nltk.FeatStruct("[A = ?x, B= [C = ?x]]")
fs2 = nltk.FeatStruct("[B = [D = d]]")
fs3 = nltk.FeatStruct("[B = [C = d]]")
fs4 = nltk.FeatStruct("[A = (1)[B = b], C->(1)]")
fs5 = nltk.FeatStruct("[A = (1)[D = ?x], C = [E -> (1), F = ?x] ]")
fs6 = nltk.FeatStruct("[A = [D = d]]")
fs7 = nltk.FeatStruct("[A = [D = d], C = [F = [D = d]]]")
fs8 = nltk.FeatStruct("[A = (1)[D = ?x, G = ?x], C = [B = ?x, E -> (1)] ]")
fs9 = nltk.FeatStruct("[A = [B = b], C = [E = [G = e]]]")
fs10 = nltk.FeatStruct("[A = (1)[B = b], C -> (1)]")

What is the result of the following unifications? 

1. `fs1` and `fs2`,  
2. `fs1` and `fs3`,  
3. `fs4` and `fs5`,  
4. `fs5` and `fs6`,  
5. `fs5` and `fs7`,  
6. `fs8` and `fs9`,  
7. `fs8` and `fs10`.  

In [17]:
#print(fs2.unify(fs1))
#print(fs1.unify(fs3))
#print(fs4.unify(fs5))
#print(fs6.unify(fs5))
#print(fs5.unify(fs7)) # share a same path but different value
#print(fs8.unify(fs9))
print(fs8.unify(fs10))

[         [ B = 'b'  ] ]
[ A = (1) [ D = 'b'  ] ]
[         [ E -> (1) ] ]
[         [ G = 'b'  ] ]
[                      ]
[ C -> (1)             ]
